# Classification analysis of airbnb review data

The API for the airbnb review open dataset for this classification analysis is available here: https://public.opendatasoft.com/explore/dataset/airbnb-reviews/api/. The goal of thei notebook is the supervised machine learning task of classifying airbnb review data for the use-case of sentiment analysis. We will use Amazon SageMaker hosting and software to this end and so we begin with the necessary imports...

In [3]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
import pandas as pd

Let's now download and unzip the reviews and listings open datasets from http://insideairbnb.com and inspect them...

In [8]:
!wget http://data.insideairbnb.com/united-kingdom/england/london/2020-04-14/data/reviews.csv.gz
!gunzip reviews.csv.gz

--2020-06-11 11:02:55--  http://data.insideairbnb.com/united-kingdom/england/london/2020-04-14/data/reviews.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.95.162
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.95.162|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179819600 (171M) [application/x-gzip]
Saving to: ‘reviews.csv.gz’

reviews.csv.gz      100%[===================>] 171.49M  13.4MB/s    in 15s     

2020-06-11 11:03:10 (11.2 MB/s) - ‘reviews.csv.gz’ saved [179819600/179819600]



In [9]:
review_dataf = pd.read_csv('reviews.csv') 
review_dataf.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [10]:
!wget http://data.insideairbnb.com/united-kingdom/england/london/2020-04-14/data/listings.csv.gz
!gunzip listings.csv.gz

--2020-06-11 11:05:31--  http://data.insideairbnb.com/united-kingdom/england/london/2020-04-14/data/listings.csv.gz
Resolving data.insideairbnb.com (data.insideairbnb.com)... 52.216.228.74
Connecting to data.insideairbnb.com (data.insideairbnb.com)|52.216.228.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78560292 (75M) [application/x-gzip]
Saving to: ‘listings.csv.gz’

listings.csv.gz     100%[===================>]  74.92M  12.5MB/s    in 6.7s    

2020-06-11 11:05:38 (11.2 MB/s) - ‘listings.csv.gz’ saved [78560292/78560292]



In [11]:
listings_dataf = pd.read_csv('listings.csv') 
listings_dataf.head()

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20200414180850,2020-04-16,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,...,f,f,moderate,f,f,2,1,1,0,0.18
1,15400,https://www.airbnb.com/rooms/15400,20200414180850,2020-04-16,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,...,t,f,strict_14_with_grace_period,t,t,1,1,0,0,0.71
2,17402,https://www.airbnb.com/rooms/17402,20200414180850,2020-04-15,Superb 3-Bed/2 Bath & Wifi: Trendy W1,You'll have a wonderful stay in this superb mo...,"This is a wonderful very popular beautiful, sp...",You'll have a wonderful stay in this superb mo...,none,"Location, location, location! You won't find b...",...,t,f,strict_14_with_grace_period,f,f,15,15,0,0,0.38
3,17506,https://www.airbnb.com/rooms/17506,20200414180850,2020-04-16,Boutique Chelsea/Fulham Double bed 5-star ensuite,Enjoy a chic stay in this elegant but fully mo...,Enjoy a boutique London townhouse bed and brea...,Enjoy a chic stay in this elegant but fully mo...,business,Fulham is 'villagey' and residential – a real ...,...,f,f,strict_14_with_grace_period,f,f,2,0,2,0,NaN
4,25023,https://www.airbnb.com/rooms/25023,20200414180850,2020-04-15,All-comforts 2-bed flat near Wimbledon tennis,"Large, all comforts, 2-bed flat; first floor; ...",10 mins walk to Southfields tube and Wimbledon...,"Large, all comforts, 2-bed flat; first floor; ...",none,This is a leafy residential area with excellen...,...,t,f,moderate,f,f,1,1,0,0,0.70


In [30]:
print(listings_dataf.columns[64],listings_dataf.columns[65],listings_dataf.columns[94],listings_dataf.columns[95])

('cleaning_fee', 'guests_included', 'license', 'jurisdiction_names')


Probably won't use these offending columns so just drop them...

In [31]:
listings_dataf = listings_dataf.drop(['cleaning_fee', 'guests_included', 'license', 'jurisdiction_names'],axis=1)